# Titulo

- Ana Margarida Ruivo Loureiro - up201705749

- João Martins

- João Ricardo Cardoso

## Abstract

## Introdução

## Dataset

O nosso dataset providencia-nos dados geográficos e contagem de casos confirmados, óbitos e recuperados do Covid-19 num periodo de  4 meses (22 de janeiro de 2020 a 5 de maio de 2020) em vários países. O objetivo será extrair informação dos dados de forma a podermos construir um modelo de regressão capaz de prever as contagens tendo por base os fatores disponiveis (localização geográfica, dia e contagem anterior). Assim sendo é necessário partir dos dados iniciais e proceder a um pré-processamento para tratar os dados e poder-se então utlizar os algoritmos de regressão que pretendemos utilizar.

### Variáveis independentes

Como foi supramencionado, pretendemos utilizar certos fatores para fundamentarmos a nossa previsão (partindo dos dados do dataset). Estes então serão as coordenadas geográficas (Latitude e Longitude), número de dias desde o inicio da previsão (ou seja, 22 de janeiro) e contagens anteriores (número de casos confirmados, óbitos e recuperados do dia anterior).

### Variáveis Dependentes

Os valores que vamos prever serão os números de casos confirmados, óbitos e recuperados, sendo então estas as nossas variáveis independentes.

## Tratamento de dados

Para podermos obter resultados fiáveis, é preciso haver um tratamento dos dados que vem do dataset.
Pegando dos dados processados anteriormente feito inicialmente, começamos por extrai-los do ficheiro para podermos manipulá-los.

In [1]:
import pandas as pd

covid_data = pd.read_csv('covid_19_clean_complete.csv')
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27451,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27452,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27453,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27454,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


De seguida, após uma análise, verificou-se a presença de entrada respeitantes a navios que nalgum momento tiveram casos de Covid-19 e não estão portanto associados a nenhum país particular.
Além disso, consideramos que estes dados iriam criar ruído, pelo que optamos por ignorá-los e remover dos dados em análise.

In [2]:
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Grand Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='MS Zaandam'].index)
covid_data = covid_data.reset_index()
del covid_data['index']
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27035,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27036,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27037,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27038,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


A maioria das entradas da coluna **Province/State** tem valores nulos, pelo que procedemos a eliminá-los. Além disso, como ter uma entrada para uma região e país não é muito relevante, optamos por agregar as duas informações numa só coluna denominada de **Local**.

In [3]:
import numpy as np

covid_data['Province/State'] = covid_data.replace(np.nan, '', regex=True)
cols = ['Province/State', 'Country/Region']
covid_data['Local'] = covid_data[cols].apply(lambda row: ' / '.join(row.values.astype(str)) if row.values[0] != '' else ''.join(row.values.astype(str)), axis=1)
del covid_data['Province/State']
del covid_data['Country/Region']
covid_data

,Lat,Long,Date,Confirmed,Deaths,Recovered,Local
0,33.000000,65.000000,1/22/20,0,0,0,Afghanistan
1,41.153300,20.168300,1/22/20,0,0,0,Albania
2,28.033900,1.659600,1/22/20,0,0,0,Algeria
3,42.506300,1.521800,1/22/20,0,0,0,Andorra
4,-11.202700,17.873900,1/22/20,0,0,0,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,5/4/20,6,0,5,Western Sahara
27036,0.186360,6.613081,5/4/20,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,5/4/20,12,2,0,Yemen
27038,-11.645500,43.333300,5/4/20,3,0,0,Comoros


De seguida, vamos converter as datas em contagem de dias desde o início do dataset (22 de janeiro de 2020)

In [4]:
covid_data['Date'] = pd.to_datetime(covid_data['Date'],format='%m/%d/%y')
covid_data['Date'] -= pd.to_datetime("2020-01-22")
covid_data['Date'] /= np.timedelta64(1,'D')
covid_data = covid_data.rename(columns  = {'Date':'Days Passed'})
covid_data

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local
0,33.000000,65.000000,0.0,0,0,0,Afghanistan
1,41.153300,20.168300,0.0,0,0,0,Albania
2,28.033900,1.659600,0.0,0,0,0,Algeria
3,42.506300,1.521800,0.0,0,0,0,Andorra
4,-11.202700,17.873900,0.0,0,0,0,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,103.0,6,0,5,Western Sahara
27036,0.186360,6.613081,103.0,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,103.0,12,2,0,Yemen
27038,-11.645500,43.333300,103.0,3,0,0,Comoros


Por fim, adicionar as colunas da contagem do dia anterior.

In [5]:
covid_data['Confirmed Before'] = covid_data.apply(lambda row: 
                                                      covid_data.loc[(covid_data['Local'] == row['Local']) & (covid_data['Days Passed'] == row['Days Passed']-1).item(), 'Confirmed']
                                                      if row['Days Passed'] > 0 else 0,axis=1)
covid_data

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
with pd.ExcelWriter('covid_19_distance.xlsx') as writer:
    covid_data.to_excel(writer)

## K Nearest Neighbours
Este dataset que processamos é ideal para se usar KNN com as coordenadas geográficas para distinguir os vizinhos ao epicentro original da pandemia (Província de Hubei, na China).
Já que pretendemos construir um modelo de regressão destes dados, vamos recorrer à ferramenta *scikit-learn* para utilizar os seus algoritmos de regressão, nos quais se encontra o **KNeighborsRegressor**, que será o escolhido para aplicar o KNN.

### KNeighborsRegressor
#### Parâmetros da pesquisa
* **n_neighbors**: número de vizinhos para usar, por defeito, 5.
* **weights**: função de peso utilizado na previsão. Valores a testar:
    * *uniform*: pesos uniformes, todos os pontos na vizinhança são pesados igualmente
    * *distance*: pesos influenciados pela distância ao ponto de pesquisa, em que pontos vizinhos mais pertos do ponto de pesquisa terão mais influência do que aqueles mais afastados
* **algorithm**: algoritmo usado para computar os nearest neighbors. Opção a usar:
    * *auto*: tenta decidir qual a melhor escolha, tendo em conta os valores passados para a função de **fit()** dentro das possíveis escolhas, *ball_tree*, *kd_tree* ou *brute*
* **metric**: definir que tipo de distância a utilizar. Métrica a testar:
    * *euclidean*: utilizar a distância euclideana (mais apropriado)
* **n_jobs**: definir nº de processos para paralelizar os trabalhos
    * *None*: não há paralelismo (usar em debug)
    * *-1*: todos os cpu's são usados

In [ ]:
#data de teste: 4/5/20
#criar set de treino 
train = covid_data[(covid_data['Date']=='5/4/20') & (~covid_data['Local'].str.contains('China'))]
train

In [ ]:
#criar set de teste
test = covid_data[(covid_data['Date']=='5/4/20') & (covid_data['Local'].str.contains('China'))]
test

In [2]:
#colunas em que vamos basear as previsões
x_columns = ['Lat','Long']
#colunas que queremos 
y_columns = ['Confirmed','Deaths','Recovered']

from sklearn.neighbors import KNeighborsRegressor
#criar o modelo e usar o número de vizinhos default, 5.
knn = KNeighborsRegressor(n_neighbors=5,weights='distance')
#aplicar a função de fit ao set de treino
knn.fit(train[x_columns],train[y_columns])
#fazer previsões do set de teste usando 
predictions = knn.predict(test[x_columns])

#tratar dos resultados
predictions = pd.DataFrame(data=predictions,columns=['Confirmed Pred.','Deaths Pred.','Recovered Pred.'])
predictions['Local'] = test['Local'].tolist()
#arredondar os valores para inteiros, com teto
predictions['Confirmed Pred.'] = predictions['Confirmed Pred.'].apply(np.ceil)
predictions['Deaths Pred.'] = predictions['Deaths Pred.'].apply(np.ceil)
predictions['Recovered Pred.'] = predictions['Recovered Pred.'].apply(np.ceil)
predictions['Confirmed Actual'] = test['Confirmed'].tolist()
predictions['Deaths Actual'] = test['Deaths'].tolist()
predictions['Recovered Actual'] = test['Recovered'].tolist()
predictions['Date'] = '5/4/20'
predictions = predictions[['Date','Local', 'Confirmed Pred.', 'Confirmed Actual','Deaths Pred.', 'Deaths Actual','Recovered Pred.','Recovered Actual']]
predictions

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
covid_data['Date']

## SVR

In [5]:
from sklearn.svm import SVR

In [6]:
covid_data

,Lat,Long,Days Passed,Confirmed,Deaths,Recovered,Local
0,33.000000,65.000000,0.0,0,0,0,Afghanistan
1,41.153300,20.168300,0.0,0,0,0,Albania
2,28.033900,1.659600,0.0,0,0,0,Algeria
3,42.506300,1.521800,0.0,0,0,0,Andorra
4,-11.202700,17.873900,0.0,0,0,0,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,103.0,6,0,5,Western Sahara
27036,0.186360,6.613081,103.0,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,103.0,12,2,0,Yemen
27038,-11.645500,43.333300,103.0,3,0,0,Comoros


Extrair variáveis dependentes e independentes do dataset

In [7]:
X = covid_data.iloc[:,0:3].values

In [8]:
Y = covid_data.iloc[:,3:6].values

Escalar os valores existentes de forma a que não sejam discrepantes uns dos outros

In [9]:
from sklearn.preprocessing import StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
Y = scaler_y.fit_transform(Y)

In [10]:
X= X.reshape(-1, 1)
Y= Y.reshape(-1, 1)

In [11]:
regressor = SVR(kernel='rbf')


In [12]:
regressor.fit(X,Y.ravel())

SVR()

In [14]:
pre = regressor.predict(X)

In [17]:
pre = pd.DataFrame(data=pre,columns=['Confirmed Pred.','Deaths Pred.','Recovered Pred.'])
pre['Local'] = test['Local'].tolist()
#arredondar os valores para inteiros, com teto
pre['Confirmed Pred.'] = pre['Confirmed Pred.'].apply(np.ceil)
pre['Deaths Pred.'] = pre['Deaths Pred.'].apply(np.ceil)
pre['Recovered Pred.'] = pre['Recovered Pred.'].apply(np.ceil)
pre['Confirmed Actual'] = test['Confirmed'].tolist()
pre['Deaths Actual'] = test['Deaths'].tolist()
pre['Recovered Actual'] = test['Recovered'].tolist()
pre['Date'] = '5/4/20'
pre = pre[['Date','Local', 'Confirmed Pred.', 'Confirmed Actual','Deaths Pred.', 'Deaths Actual','Recovered Pred.','Recovered Actual']]
pre

ValueError: Shape of passed values is (81120, 1), indices imply (81120, 3)